## 七月在线机器学习第九期第二阶段考试
#### 考试说明:
- 起止时间：请同学在2018年7月11日至7月18日期间完成，最晚提交时间（7月18日24时之前）结束，<b>逾期不接受补考,该考试分数计入平时成绩</b>
- 考试方式：请同学将该试卷进行复制后，改名为自己姓名后，如State2Exam-WangWei.ipynb，<b>移动</b>至/0.Teacher/Exam/Stage2/目录下后，进行答题。
- 注意事项：为确保同学们真正了解自身对本周课程的掌握程度，<font color=red><b>请勿翻阅，移动，更改</b></font>其它同学试卷。如发现按0分处理
- 请同学在下方同学姓名处填写自己的姓名，批改人和最终得分不用填写

- 同学姓名:MichelleYang
- 批改人：   
- 最终得分:

<center><h1>####答卷开始####</h1></center>
<hr>

## 问答题(共5题，每题10分，共计50分)

#### 1. 请写出你了解的机器学习特征工程操作都有哪些，及它们的具体实现？

In [ ]:
特征工程主要是从数据中抽取出对结果预测有用的信息。特征工程主要涵盖数据采集、数据处理、特征处理、特征选择或降维等操作。
数据采集模块中，需要考虑数据对最后结果预测的影响力，即所谓的事先考虑数据和最后预测结果的关联度；考虑数据采集的难易程度等。
数据处理常见的操作是数据清洗以及数据采样。数据清洗主要是去除脏数据、补齐缺省值（中位数、平均数、众数补齐或者是通过模型预测）等。
数据采样主要针对的是数据若不均衡，而有些模型对数据比较敏感，则会通过采样、SMOTE生成新数据、修改损失函数以及收集更多数据等方式来弥补。
特征处理，数据处理后的特征针对不同的数据的类别分别进行不同的操作，针对数值型数据可以通过归一化、标准化进行幅度调整或者是将数据等频或者等距离散化。
针对类别型数据进行one-hot编码等；针对文本型数据可以映射成稀疏向量；组合特征等一系列的特征处理操作。
特征选择主要是一些特征冗余，可以剔除掉特征。主要方式主要有三类：filter（评估单个特征和结果的相关度）、wrapper（子集搜索）、embedding等。
特征降维主要是对特征进行计算组合构成新的特征，例如PCA等。


#### 2.模型评估中的留一法，留出法，交叉验证分别是什么操作？

In [ ]:
留一法主要是将N个样本分成N份，N-1个样本做训练集，1个样本做交叉验证集，总共进行N次交叉验证。
留出法主要是将数据分为训练集和测试集，常见的训练集与训练集样本数量的比值是4:1或者是7:3.
K折交叉验证主要是将N个训练样本分为k份，k-1份做训练，1份进行交叉验证，总共进行k次交叉验证。

#### 3.网格搜索交叉验证的作用是什么，并简述网格搜索交叉验证是如何运行的？

In [ ]:
网格搜索交叉验证主要是用来通过候选集调超参数的，是微调模型的一种方法。
网格交叉验证，正如名称一样，不同的超参数的取值进行组合，来寻找在当前候选集中最优的超参数组合。

#### 4.工业界上模型融合有三大类方式？试简述每类方式其思想？

In [ ]:
bagging：采用的是一个算法，对数据进行有放回的方式进行抽样，用抽样的样本建立子模型,对子模型进行训练，这个过程重复多次，最后进行融合
stacking：用多种分类器结果作为特征训练
boosting：简单的分类器的叠加，分配给分错的样本更高的权重

#### 5.Early Stopping指的是什么？与它相关的概念有哪些？它们在实际项目中如何被运用？

In [ ]:
随着epoch增加，在数据集不断进行训练，检测测试集，如果在测试集上发现测试误差上升或者不变，则停止训练，为了防止过拟合。

## 代码题(共1题，共计50分)

#### 1.使用XGBoost的sklearn接口，对KaggleCredit2数据完成信用卡欺诈项目的建模及分析
- 要求以不同参数设置xgboost运行并进行效果比对,记录，最后给出实验报告

- KaggleCredit2数据文件 位于/home/ml9/0.Teacher/Data，请勿复制或移动

In [ ]:
# -*- coding:UTF-8 -*-
# author: michelle
# time: 2018/7/18 14:43
# -*- coding:UTF-8 -*-
# author: michelle
# time: 2018/7/18 12:16
import pandas as  pd
import matplotlib.pyplot as plt
import numpy as np

# 数据预处理与模型选择
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold

from sklearn.grid_search import  GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
import itertools

# 随机森林与SVM

from scipy import stats
from xgboost.sklearn import XGBClassifier

show_bdry = False
show_best_c = False
# 对所有特征做标准化处理，或者对Amount做
def normalize_feature(data, amount_only = False):
    if amount_only:
        data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))
    else:
        for feature in data.columns[:-1]:
            data[feature] = StandardScaler().fit_transform(data[feature].values.reshape(-1,1))
    return data


def split_train_test(fraud_indices, normal_indices, test_size = 0.3):
    number_records_fraud = len(fraud_indices)
    number_records_normal = len(normal_indices)
    test_fraud_end = int(number_records_fraud * test_size)
    test_normal_end = int(number_records_normal  * test_size)

    test_fraud_indices = fraud_indices[0:test_fraud_end]
    train_fraud_indices = fraud_indices[test_fraud_end:]

    test_normal_indices = normal_indices[0:test_normal_end]
    train_normal_indices = normal_indices[test_normal_end:]

    return train_normal_indices, train_fraud_indices, test_normal_indices, test_fraud_indices

# 训练集做下采样
def getTrainingSample(train_fraud_indices, train_normal_indices, data, train_normal_pos, ratio):
    train_number_records_fraud = int(ratio*len(train_fraud_indices))
    train_number_records_normal = len(train_normal_indices)
    if train_normal_pos + train_number_records_fraud <= train_number_records_normal:
        small_train_normal_indices = train_normal_indices[train_normal_pos: train_normal_pos + train_number_records_fraud]
        train_normal_pos = train_normal_pos + train_number_records_fraud
    else:
        small_train_normal_indices = np.concatenate([train_normal_indices[train_normal_pos: train_number_records_normal],train_normal_indices[0: train_normal_pos + train_number_records_fraud- train_number_records_normal]])
        train_normal_pos = train_normal_pos + train_number_records_fraud - train_number_records_normal

    under_train_sample_indices = np.concatenate([train_fraud_indices, small_train_normal_indices])
    np.random.shuffle(under_train_sample_indices)
    # 下采样
    under_train_sample_data = data.iloc[under_train_sample_indices,:]

    X_train_undersample = under_train_sample_data.ix[:, under_train_sample_data.columns != 'Class']
    y_train_undersample = under_train_sample_data.ix[:, under_train_sample_data.columns == 'Class']

    return X_train_undersample, y_train_undersample, train_normal_pos

def compute_recall_and_auc(y_t, y_p):
    # 混淆矩阵
    cnf_matrix = confusion_matrix(y_t,y_p)
    np.set_printoptions(precision = 2)
    recall_score = cnf_matrix[1,1]/(cnf_matrix[1,0] + cnf_matrix[1,1])

    # ROC曲线与auc
    fpr, tpr, thresholds = roc_curve(y_t, y_p)
    roc_auc = auc(fpr, tpr)
    return recall_score, roc_auc


def run(data, mode, ratio, iteration1):
    for itr1 in range(iteration1):
        #print("percentage: %.2f" %(itr1/iteration1*100))

        # 欺诈类的样本
        fraud_indices = np.array(data[data.Class == 1].index)
        np.random.shuffle(fraud_indices)

        # 正常类的样本
        normal_indices = np.array(data[data.Class == 0].index)
        np.random.shuffle(normal_indices)

        # 训练集与测试集
        train_normal_indices, train_fraud_indices, test_normal_indices, test_fraud_indices = split_train_test(
                                                                                            fraud_indices, normal_indices)
        test_indices = np.concatenate([test_normal_indices,test_fraud_indices])

        test_data = data.iloc[test_indices,:]
        X_test = test_data.ix[:, test_data.columns != 'Class']
        y_test = test_data.ix[:, test_data.columns == 'Class'].values.ravel()

        # 数据下采样
        X_train_undersample, y_train_undersample, train_normal_pos = getTrainingSample(
                                                                    train_fraud_indices, train_normal_indices, data, 0, ratio)

        # 训练模型
        param_test1 = {
            'max_depth': list(range(3, 10, 2)),
            'min_child_weight': list(range(1, 6, 2))
        }
        model_xgb = GridSearchCV(estimator=XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                                                        min_child_weight=1, gamma=0, subsample=0.8,
                                                        colsample_bytree=0.8, objective='binary:logistic', nthread=4,
                                                        scale_pos_weight=1, seed=27),
                                param_grid=param_test1, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
        model_xgb.fit(X_train_undersample.values, y_train_undersample.values.ravel())

        # 预测
        y_pred = model_xgb.predict(X_test.values)
        # 记录指标
        recall_score, roc_auc = compute_recall_and_auc(y_test, y_pred)
    return recall_score,roc_auc
if __name__=="__main__":
    mode = 2
    ratio = 1
    iteration1 = 1

    data = pd.read_csv("./creditcard.csv")
    data = data.drop(['Time'], axis = 1)
    data = normalize_feature(data, amount_only = True)

    # 总体开始跑
    result, control= run(data = data, mode = mode, ratio = ratio, iteration1 = 1)
    print(result,control)


***
<center><h1>####答卷结束####</h1></center>